In [1]:
from src.llm_plan.planner import Planner
from src.llm_plan.llm import GPT_4o

specific = "A single agent blocks world planner that can only move one block at a time. The blocks are labeled with single uppercase letters (A, B, C, etc.). The planner should be able to handle the following actions: 'pick up', 'put down', 'stack', and 'unstack'. The planner should also be able to handle the following conditions: 'on', 'clear', 'holding', and 'table'. The planner should be able to generate a sequence of actions to achieve a given goal state from a given initial state."
environment_name = "BlocksWorld"

model = GPT_4o()
planner = Planner(model)

In [2]:
planner.generate_representation(specific, environment_name ,"json")

Here's an example of a json file that describes an environment:
<environment-json>{'name': 'TwoAgentsVault', 'author': 'Human', 'agents': {'number': 3, 'names': ['key_holder', 'small_robot', 'orchestrator'], 'key_holder': {'private_information': ['I have a key', 'I am a big robot'], 'goal': 'Open the vault and grab the object inside.'}, 'small_robot': {'private_information': ['I am a small robot'], 'goal': 'Open the vault and grab the object inside.'}, 'orchestrator': {'private_information': [], 'goal': 'Open the vault and grab the object inside.'}}, 'environment': {'init': {'grid_size': 4, 'visibility': 1}, 'public_information': ['There is a vault in the environment.', 'The vault is closed.', 'The vault requires a key to be opened.', 'The entrance of the vault is small.']}, 'workflow': {'key_holder': {'pddl': {'input': [], 'output': 'pddl_key_holder', 'system_prompt': 'You are an expert with PDDL problems (Planning Domain Definition Language). You always provide a PDDL domain and a PD

PosixPath('environments/static/BlocksWorld')

In [ ]:
from src.llm_plan.environment import Environment

env = Environment("./environments/static/TwoAgentsVault.json")
env.plan

In [ ]:
# Plan
responses = planner.plan()

In [ ]:
# Extract the pddl problem and domain from the LLM response
from src.llm_plan.parser import PDDLParser

pddl_parser = PDDLParser()

# TODO: fix the vocabulary entry, it shouldn't be "pddl_orchestrator"
final_plan = responses["pddl_orchestrator"]
domain, problem = pddl_parser.parse(final_plan, from_file=False)

In [ ]:
# Obtain the plan with the solver
import subprocess

from pathlib import Path

from src.llm_plan.config import SOLVER_BINARY, SOLVER_ARGS

BASE_FOLDER = Path(f"./tmp/{env.name}")
BASE_FOLDER.mkdir(parents=True, exist_ok=True)

with open(BASE_FOLDER / "problem.pddl", "w") as f:
    f.write(problem)

with open(BASE_FOLDER / "domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / "sas_plan",
    BASE_FOLDER / "domain.pddl",
    BASE_FOLDER / "problem.pddl",
]

with open(BASE_FOLDER / "logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)

In [ ]:
# Validate the plan with uVAL
import subprocess

from src.llm_plan.config import UNIVERSAL_VALIDATOR_BIN

command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / 'domain.pddl'} \
{BASE_FOLDER / 'problem.pddl'} \
{BASE_FOLDER / 'sas_plan'}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    pddl_error = out.stderr
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    pddl_error = "No error found."
    print("The plan is valid.")
    print("[stdout]", out.stdout)

In [ ]:
# Let's try to refine the plan syntax
from src.llm_plan.hypervisor import HypervisorDeepThinkPDDL, HypervisorSyntaxPDDL

num_attempts = 5

for i in range(num_attempts):
    
    # General validator
    prompt_args = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
    }

    deep_think = HypervisorDeepThinkPDDL(llm=model, prompt_args=prompt_args)

    r = deep_think.run()
    domain, problem = pddl_parser.parse(r, from_file=False)
    
    # Obtain the new plan with the solver
    with open(BASE_FOLDER / "refined_problem.pddl", "w") as f:
        f.write(problem)

    with open(BASE_FOLDER / "refined_domain.pddl", "w") as f:
        f.write(domain)
        
    # Launch the solver
    command = [
        SOLVER_BINARY,
        *SOLVER_ARGS,
        BASE_FOLDER / "refined_sas_plan",
        BASE_FOLDER / "refined_domain.pddl",
        BASE_FOLDER / "refined_problem.pddl",
    ]

    with open(BASE_FOLDER / "refined_logs.txt", "w") as logfile:
        subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
        
    command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
    {BASE_FOLDER / 'refined_domain.pddl'} \
    {BASE_FOLDER / 'refined_problem.pddl'} \
    {BASE_FOLDER / 'refined_sas_plan'}"

    out = subprocess.run(command, shell=True, capture_output=True, text=True)

    # This part won't be printed at test time
    if out.stderr:
        pddl_error = out.stderr
        print(
            f"The validation found a problem with the plan: {out.stderr}"
        )
    else:
        pddl_error = "No error was found."
        print("The plan is valid.")
        print("[stdout]", out.stdout)
    
    # Syntax validator
    print(f"Validation attempt {i+1}/{num_attempts}")
    prompt_args = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
        "syntax_errors": pddl_error,
    }

    hypervisor_pddl = HypervisorSyntaxPDDL(llm=model, prompt_args=prompt_args)

    r = hypervisor_pddl.run()
    domain, problem = pddl_parser.parse(r, from_file=False)

In [ ]:
# Final plan run and validation
with open(BASE_FOLDER / "refined_problem.pddl", "w") as f:
    f.write(problem)

with open(BASE_FOLDER / "refined_domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / "refined_sas_plan",
    BASE_FOLDER / "refined_domain.pddl",
    BASE_FOLDER / "refined_problem.pddl",
]

with open(BASE_FOLDER / "refined_logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
    
command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / 'refined_domain.pddl'} \
{BASE_FOLDER / 'refined_problem.pddl'} \
{BASE_FOLDER / 'refined_sas_plan'}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    pddl_error = out.stderr
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    pddl_error = "No error found."
    print("The plan is valid.")
    print("[stdout]", out.stdout)

In [ ]:
# Report the natural language plan
from src.llm_plan.hypervisor import HypervisorNaturalLanguage

with open(BASE_FOLDER / "refined_problem.pddl", "r") as f:
    problem = f.read()

with open(BASE_FOLDER / "refined_domain.pddl", "r") as f:
    domain = f.read()
    
with open(BASE_FOLDER / "refined_sas_plan", "r") as f:
    plan = f.read()
    
prompt_args = {
    "specification": env.config_data,
    "pddl_domain": domain,
    "pddl_problem": problem,
    "pddl_plan": plan,
}

hypervisor_to_nl = HypervisorNaturalLanguage(llm=model, prompt_args=prompt_args)

natural_plan = hypervisor_to_nl.run()

with open(BASE_FOLDER / "refined_nl_plan.txt", "w") as f:
    f.write(natural_plan)